# NLP I - Exercise 2 solution

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import spacy
nlp = spacy.load('es_core_news_sm')

import urllib.request # Hacer peticiones HTML
from bs4 import BeautifulSoup # Manipular y trabajar con HTML

In [3]:
def es_Spam(url_email):
    texto = obtener_texto(url_email, 'p')
    texto = texto + obtener_texto(url_email, 'a')
    
    densidad = text_density(texto, True)
    
    palabras_spam_1 = ['comprar', 'descargar', 'descuento', 'ayuda', 'regalo', 'premio', 'bancaria', 'más']
    palabras_spam = []
    
    for token in nlp(' '.join(palabras_spam_1)):
            palabras_spam.append(token.lemma_) # Añadimos a lista
    
    spam_points = 0
    
    fila = 0
    filas = densidad.shape[0]
        
    while fila < filas:
        
        row = densidad.iloc[fila]
        
        if row.loc['Palabra'] in palabras_spam:
            
            if float(row.loc['Densidad']) > 0.02:
                
                spam_points = spam_points + 1
        
        fila = fila +1
    
    result = ''
    
    if spam_points > 0:
        result = 'Si es SPAM'
    else:
        result = 'No es SPAM'
         
    return result + ' Puntos Spam: ' + str(spam_points)

In [4]:
def text_density(content, clean):
    
    # 1.- Tokenizar
    tokens = word_tokenize(content, "spanish")
    # Elimnar puntuación
    tokens_alpha = [word.lower() for word in tokens if word.isalpha()] 
    
    # 2.- Eliminar Stopwords
    clean_tokens = tokens_alpha[:] # Duplicamos los tokens
    
    for token in tokens_alpha:
        if token in stopwords.words('spanish'):
            clean_tokens.remove(token)
    
    # 3.- Función para limpiar y obtener lemas
    cleaned_tokens = [] # Array
    
    if clean:
        # Realizamos la limpieza (SPACY)
        separator = ' ' # Siempre es un espacio
        
        # Recorremos cada token analizado por spacy
        for token in nlp(separator.join(clean_tokens)):
            cleaned_tokens.append(token.lemma_) #Añadimos a lista
    else:
        cleaned_tokens = clean_tokens
        
    # 4.- Generamos la tabla para el output
    conteo = pd.DataFrame(columns = ['Palabra', 'Recuento', 'Densidad'])
    
    # 5.- Obtenemos los datos para llenar la tabla
    freq = nltk.FreqDist(cleaned_tokens) # Usamos NLTK
    
    #Recorremos el análisis de NKT y frecuencia
    for key,val in freq.items():
        
        # Key es el token
        # Val es el número de repeticiones de ese token
        
        # En este blucle insertamos las columnas
        dens = "{:.2}".format(val/len(cleaned_tokens)) # Obtenemos densidad
        reps = "{:.0f}".format(val) # Obtenemos repeticiones
        
        fila = pd.Series([key, reps, dens], index = conteo.columns)
        conteo = conteo.append(fila, ignore_index=True)
        
    result = conteo.sort_values('Densidad', ascending=False)
    
    return result

In [5]:
def obtener_texto(url_pet, elemento):
    
    # 1.- Hacemos la petición a la web y descargamos el HTMLt
    request = urllib.request.urlopen(url_pet)
    
    # 2.- Leemos el HTML
    html = request.read()
    
    # 3.- Parsear el HTML (Utilizando Beautiful Soup)
    soup = BeautifulSoup(html, 'html.parser')
    
    # 4.- Recuperar un elemento concreto de la web
    parrafos = soup.find_all(elemento)
    
    # 5.- Crear la respuesta, obteniendo solo el texto del elemento
    result = ''
    for parrafo in parrafos:
        result = result + parrafo.get_text()
        
    return result

In [6]:
email1 = 'https://vgwq.campaign-view.com/ua/viewinbrowser?od=3z76be2304fc1bc51a1bd72c445d4b63701474d74f8db12551960783c3e1a78592&rd=11c4aa5bc740ea1a&sd=11c4aa5bc74aa5db&n=11699e4c2c382ee&mrd=11c4aa5bc74aa5c5&m=1'
email2 = 'http://view.mail.360imprimir.es/?qs=240379cda28af83ad8e10007f41fbdc1f01d1b478296dce186feaa145a49a34f23b8a8237db5b919c795c23a8f6d5674a7029b3ea6825c6d72b6ff6ce79263a31674a89eff62e90f6a1f9f8b7debe9ca'

print('Email 1: ' + es_Spam(email1))
print('Email 2: ' + es_Spam(email2))


Email 1: Si es SPAM Puntos Spam: 1
Email 2: No es SPAM Puntos Spam: 0
